In [87]:
import numpy as np
import itertools
from functools import reduce
import random
import secrets

In [126]:
def bytesToArray(targetBytes, blockSize):
    binString = ''
    for utfInt in targetBytes:
        binString += format(utfInt, '08b')

    blockedBin = [''.join(x) for x in itertools.zip_longest(*[iter(binString)]*blockSize, fillvalue='0')]

    blockedBinArr = np.zeros(shape=(len(blockedBin), blockSize), dtype=np.int8)

    for i in np.arange(len(blockedBin)):
        for j in np.arange(blockSize):
            blockedBinArr[i][j] = int(blockedBin[i][j])

    return blockedBinArr

In [127]:
def arrayToBytes(targetArr, blockSize, decoding):
    #array holding indicies which will be saved for parity bits
    parityIndicies = 2**np.arange(int(np.log2(blockSize)) - 1, -1, -1)

    #read non-parity bits into a string
    result = ''
    for block in targetArr:
        for i, bit in enumerate(block):
            if decoding:
                if i not in parityIndicies and i:
                    result += str(bit)
            else:
                result += str(bit)
                
    #split binary string into binary bytes and return the result as a bytearray
    splitResult = [''.join(x) for x in zip(*[iter(result)]* 8)]
    final = []
    for bin in splitResult:
        final.append(int(bin,2))

    return bytes(final)

In [128]:
def encode(targetBytes, blockSize):
    #array holding indicies which will be saved for parity bits
    parityIndicies = 2**np.arange(int(np.log2(blockSize)) - 1, -1, -1)

    dataLen = blockSize - int(np.log2(blockSize)) - 1
    blockedBin = bytesToArray(targetBytes, dataLen)

    #create array to hold coded blocks
    blockNum = len(blockedBin)
    formattedBlockedBin = np.zeros(shape=(len(blockedBin), blockSize), dtype=np.int8)

    #step through blockedBin and insert values into formattedBlockedBin
    for block in np.arange(blockNum): 
        formIdx = 3
        for char in blockedBin[block]:
            while formIdx in parityIndicies:
                formIdx += 1
            formattedBlockedBin[block, formIdx] = int(char)
            formIdx += 1
         
    #step through formatted blocks and set parity bits, then randomy flip a non parity bit
    parityKeyLen = str(len(parityIndicies))
    for block in formattedBlockedBin:
        locations = [i for i, bit in enumerate(block) if bit]
        parityKey = format(reduce(lambda x, y: x^y, locations), '0'+parityKeyLen+'b') ##THIS IS WHY IT WONT WORK WITH NUMBERS OTHER THAN 16, NEED TO FIX PARITYINDICIES
        for i, bit in enumerate(parityKey):
            if int(bit):
                block[parityIndicies[i]] = 1

        randomIdx = secrets.randbelow(blockSize)
        while randomIdx in parityIndicies:
            randomIdx = secrets.randbelow(blockSize)
        block[randomIdx] = 0 if block[randomIdx] else 1
    
    return arrayToBytes(formattedBlockedBin, blockSize, False)

In [129]:
#dicode the flipped bit with xor
def decode(targetBytes, blockSize):
    blockedBin = bytesToArray(targetBytes, blockSize)

    for block in blockedBin:
        locations = [i for i, bit in enumerate(block) if bit]
        wrongBit = reduce(lambda x, y: x^y, locations)
        block[wrongBit] = 0 if block[wrongBit] else 1

    return arrayToBytes(blockedBin, blockSize, True)

In [130]:
test = 'Now its easier for me to test things a lot. How does it do with a longer string and some weird characters? *!#% Seems good so far'.encode()
blockSize = 32
coded = encode(test, blockSize)
decoded = decode(coded, blockSize)
coded, decoded

(b'\x04s=\xd4x\x8dWG\t\x10\x19XefIeo\x11\x81\x91S\xee2\x06=\xb2H=I\xde\xa0vF\xab\xcd\xd2X\x0e\x06\x861w\x19\xdcT@a"\x0e\xe3=\xd2\x13\x84\x04\x86/;\xc8\x19!^\xe5wJ\x83%\xd88\x8cF\xe2\x00;\xd8]a\xd0!a*\x13\xb1\xbd{Lvwb\x92\x9c\xddId\xedn\x0e\xbd\x01\x85SL\xc2\x17!\xb7\xdbYE\xc0we.O\xc9\x90\xf0\x8c\xb6\x86\t\xb9\x98\x18\xd5\xe8erO\x19|\x90r\xc4\x92\x12\xad\x10\x94\xd9MJm\xf3J#\x1d\xbds\xac\xc2\x07C\'\xc8\x19YBr\x04',
 b'Now its easier for me to test things a lot. How does it do with a longer string and some weird characters? *!#% Seems good so far\x00')